<a href="https://colab.research.google.com/github/yceffort/pytorch-study/blob/master/02_3)_multi_layer_perception_NEWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data_path = 'gdrive/My Drive/Colab Notebooks/pytorch/data/hkib-20000-40075.tar/'

In [7]:
!pip3 install konlpy

    100% |████████████████████████████████| 19.4MB 2.7MB/s 
    100% |████████████████████████████████| 174kB 32.1MB/s 
  Running setup.py bdist_wheel for JPype1 ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /root/.cache/pip/wheels/0e/2b/e8/c0b818ac4b3d35104d35e48cdc7afe27fc06ea277feed2831a
Successfully built JPype1


In [0]:
import os
import re

from sklearn import datasets, model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from konlpy.tag import Hannanum
from konlpy.tag import Kkma

import pandas as pd
import numpy as np

In [0]:
target_dir = 'HKIB-20000'
cat_dirs = ['healths', 'economy', 'science', 'education', 'culture', 'society', 'industry', 'leisure', 'politics']
cat_prefixes = ['건강', '경제', '과학', '교육', '문화', '사회', '산업', '여가', '정치']

files = os.listdir(data_path+'/'+target_dir)

In [0]:
files

['hkib20000-cat03-file3.categories',
 'hkib20000-cat07-all.categories',
 'hkib20000-cat07-file3.categories',
 'hkib20000-cat03-file1.categories',
 'hkib20000-cat07-file4.categories',
 'hkib20000-cat07-file2.categories',
 'hkib20000-cat03-file4.categories',
 'hkib20000-cat03-file5.categories',
 'hkib20000-cat03-all.categories',
 'hkib20000-cat07-file5.categories',
 'hkib20000-cat03-file2.categories',
 'hkib20000-cat07-file1.categories',
 'HKIB-20000_003.txt',
 'HKIB-20000_002.txt',
 'HKIB-20000_001.txt',
 'HKIB-20000_005.txt',
 'HKIB-20000_004.txt']

In [0]:
# 데이터 정리
files = os.listdir(data_path+'/'+target_dir)

# 5분할된 텍스트 파일을 각각 처리
for file in files:
    # 데이터가 담긴 파일만 처리
    if not file.endswith('.txt'):
        continue
    
    # 각 텍스트 파일을 처리
    with open(data_path + 'HKIB-20000/' + file) as currfile:
        doc_cnt = 0
        docs = []
        curr_doc = None
        
        # 기사 단위로 분할하여 리스트를 생성
        for curr_line in currfile:
            if curr_line.startswith('@DOCUMENT'):
                if curr_doc is not None:
                    docs.append(curr_doc)
                curr_doc = curr_line
                doc_cnt = doc_cnt + 1
                continue
            curr_doc = curr_doc + curr_line
        
        # 각 기사를 대주제 별로 분류하여 기사별 파일로 정리
        for doc in docs:
            doc_lines = doc.split('\n')
            doc_no = doc_lines[1][9:]
            
            # 주제 추출
            doc_cat03 = ''
            for line in doc_lines[:10]:
                if line.startswith("#CAT'03:"):
                    doc_cat03 = line[10:]
                    break
            
            # 추출한 주제 별로 디렉토리 정리
            for cat_prefix in cat_prefixes:
                if doc_cat03.startswith(cat_prefix):
                    dir_index = cat_prefixes.index(cat_prefix)
                    break
                    
            # 문서 정보를 제거하고 기사 본문만 남기기
            filtered_lines = []
            for line in doc_lines:
                if not (line.startswith('#') or line.startswith('@')):
                    filtered_lines.append(line)
                    
            # 주제별 디렉토리에 기사를 파일로 쓰기
            filename = 'hkib-' + doc_no + '.txt'
            filepath = data_path + 'HKIB-20000/' + cat_dirs[dir_index]
            
            if not os.path.exists(filepath):
                os.makedirs(filepath)
            f = open(filepath + '/' + filename, 'w')
            f.write('\n'.join(filtered_lines))
            f.close()

In [0]:
# dirs = ['education', 'healths']
dirs = cat_dirs

x_ls = []
y_ls = []

tmp1 = []
tmp2 = ''

tokenizer = Kkma()

for i, d in enumerate(dirs):
  files = os.listdir(data_path+'HKIB-20000/'+d)
  
  for file in files:
    f = open(data_path+'HKIB-20000/'+d+'/'+file, 'r', encoding='UTF-8')
    raw = f.read()
    
    reg_raw = re.sub(r'[-\'@#:/◆▲0-9a-zA-Z<>!-"*\(\)]', '', raw)
    reg_raw = re.sub(r'[ ]+', ' ', reg_raw)
    reg_raw = reg_raw.replace('\n', ' ')
    
    tokens = tokenizer.nouns(reg_raw)
    
    for token in tokens:
      tmp1.append(token)
      
    tmp2 = ' '.join(tmp1)
    x_ls.append(tmp2)
    tmp1 = []

    y_ls.append(i)
    
    f.close()

In [0]:
len(x_ls)

In [0]:
len(y_ls)

In [12]:
x_array = np.array(x_ls)
y_array = np.array(y_ls)

cntvec = CountVectorizer()
x_cntvecs = cntvec.fit_transform(x_array)
x_cntarray = x_cntvecs.toarray()

pd.DataFrame(x_cntarray)

,0,1,2,3,4,5,6,7,8,9,...,33562,33563,33564,33565,33566,33567,33568,33569,33570,33571
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
tfidf_vec = TfidfVectorizer(use_idf=True)
x_tfidf_vecs = tfidf_vec.fit_transform(x_array)
x_tfidf_array = x_tfidf_vecs.toarray()

pd.DataFrame(x_tfidf_array)

,0,1,2,3,4,5,6,7,8,9,...,33562,33563,33564,33565,33566,33567,33568,33569,33570,33571
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(x_tfidf_array, y_array, test_size=0.2)

In [0]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [0]:
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).float()

In [0]:
train = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train, batch_size=100, shuffle=True)

In [0]:
# 신경망 구성
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(33572, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc5(x)
        return F.log_softmax(x)
    
# 인스턴스 생성
model = Net()

In [37]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.05)

for epoch in range(20):
  total_loss = 0
  
  for train_x, train_y in train_loader:
    train_x, train_y = Variable(train_x), Variable(train_y)
    optimizer.zero_grad()
    output = model(train_x)
    loss = criterion(output, train_y)
    loss.backward()
    optimizer.step()
    total_loss += loss.data.item()
    
  print(epoch + 1, total_loss)
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


1 38.38336819410324
2 8.909843444824219
3 8.036376178264618
4 5.0429191291332245
5 0.6321913157589734
6 0.36786121205659583
7 0.1875206055410672
8 0.09356582369946409
9 0.19905566411633657
10 0.24452137211302927
11 0.17251877791463244
12 95.35842226375826
13 9.377646308857948
14 41.74230623245239
15 9.468356981873512
16 13.890722638964142
17 3.511947676539421
18 6.387346982955933
19 0.7886966355144978
20 0.313082629814744


In [38]:
test_x, test_y = Variable(test_X), Variable(test_Y)
result = torch.max(model(test_x).data, 1)[1]
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())
accuracy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.9203980099502488